
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>


# Task 2 - Silver - Gold Table
This Notebook is used for task 2 in the job from the directions in notebook: **DEWD00 - 04-Creating a Simple Databricks Job**

## Classroom Setup

Run the following cell to configure your working environment for this course.

**NOTE:** The `DA` object is only used in Databricks Academy courses and is not available outside of these courses. It will dynamically reference the information needed to run the course.

In [0]:
%run ../Includes/Classroom-Setup-04

## Configure Your Environment

1. Set the default catalog to **getstarted** and the schema to your specific schema. Then, view the available tables to confirm that no tables currently exist in your schema.

In [0]:
%python
# Set the catalog and schema
spark.sql(f'USE CATALOG {DA.catalog_name}')
spark.sql(f'USE SCHEMA {DA.schema_name}')

### SILVER
**Objective**: Transform the bronze table and create the silver table.

1. Create a table named **current_employees_silver_job** from the **current_employees_bronze_job** table. 

    The table will:
    - Select the columns **ID**, **FirstName**, **Country**.
    - Convert the **Role** column to uppercase.
    - Add two new columns: **CurrentTimeStamp** and **CurrentDate**.

In [0]:
-- Create a temporary view to use to merge the data into the final silver table
CREATE OR REPLACE TABLE current_employees_silver_job AS 
SELECT 
  ID,
  FirstName,
  Country,
  upper(Role) as Role,                 -- Upcase the Role column
  current_timestamp() as CurrentTimeStamp,    -- Get the current datetime
  date(CurrentTimeStamp) as CurrentDate       -- Get the date
FROM current_employees_bronze_job;

### GOLD
**Objective:** Aggregate the silver table to create the final gold table.

1. Create a temporary view named **temp_view_total_roles_job** that aggregates the total number of employees by role. Then, display the results of the view.

In [0]:
CREATE OR REPLACE TEMP VIEW temp_view_total_roles_job AS 
SELECT
  Role, 
  count(*) as TotalEmployees
FROM current_employees_silver_job
GROUP BY Role;

2. Create the final gold table named **total_roles_gold_job** with the specified columns.

In [0]:
CREATE TABLE IF NOT EXISTS total_roles_gold_job (
  Role STRING,
  TotalEmployees INT
);

3. Insert all rows from the aggregated temporary view **temp_view_total_roles_job** into the **total_roles_gold_job** table, overwriting the existing data in the table. This overwrites the data in a table but keeps the existing schema and table definition and properties.

    Confirm the following:
    - **num_affected_rows** is *4*
    - **num_inserted_rows** is *4*

In [0]:
INSERT OVERWRITE TABLE total_roles_gold_job
SELECT * 
FROM temp_view_total_roles_job;

&copy; 2026 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="_blank">Apache Software Foundation</a>.<br/><br/><a href="https://databricks.com/privacy-policy" target="_blank">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use" target="_blank">Terms of Use</a> | <a href="https://help.databricks.com/" target="_blank">Support</a>